<h3> HANDY PYTHON PACKAGES </h3>

In [1]:
import requests
import json
from random import randint
import pandas as pd
from IPython.display import display

<H3> PREREQUISITES</H3>

In [2]:
api_key = '' #write you SOCIB api_key
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json',
    'api_key': api_key,
}

<h3>AUXILIARY FUNCTIONS</h3>

In [3]:
def params_concatenation(params, values):
    """
    Concatenates the params and their values in an unique string to later be attached to the generic API ENDPOINTS
    """
    params_concatenation = ''
    for param, value in zip(params,values):
        if params.index(param) != len(PARAMS)-1:
            params_concatenation = params_concatenation + param+'='+value+'&'
        else:
            params_concatenation = params_concatenation + param+'='+value
    return params_concatenation;

<h3> DATA-PRODUCTS ENDPOINT </h3>

SOCIB Data-products can either be isolated deployments (equivalent for data-sources when so) or, a group of deployments that share some context becuase they were concurrent (multiplatform monitoring operations) or recurrent(long-term monitoring operations) deployments. 

Some examples of these are the so called glider-missions, oceanographic campaings, fixed-stations etc. On those cases, the focus is on retrieveing data continously from an specific area, track or point no matter how many deployments have been done to ensure this. Deployments becomes secondary and there so grouping them in major entities becomes handy.

As we already saw in Quick start there is an ENDPOINT that allows users to dive into all the data-products that SOCIB has composed:

In [4]:
end_point = '/data-products/'
request = requests.get('%s%s' % (api_url, end_point), headers=headers)
response = json.loads(request.text)

print('Requested url: '+'%s%s' % (api_url, end_point))
print(' ')
print('SOCIB has composed so far a total of %s data-products out of deployments'%(response['count']))

Requested url: http://api.socib.es/data-products/
 
SOCIB has composed so far a total of 157 data-products out of deployments


Choosing randomly one of the above data-products:

In [79]:
random = randint(0,response['count']-1) #random deployment
print(random)

148


In [80]:
page = random/8+1
item = (random - (random/8)*8-1)
print('Data-product number %s is on page %s at position %s'%(random+1,page,item))

Data-product number 149 is on page 19 at position 3


In [81]:
#requesting the page where the deployment is
end_point = '/data-products/'
random_request = requests.get('%s%s?%s' % (api_url, end_point, 'page='+str(page)), headers=headers)
random_response = json.loads(random_request.text)
print('Requested url: '+'%s%s?%s' % (api_url, end_point, 'page='+str(page)))
print('')
print('Data-product on page %s at position %s is described by at least 3 properties:'%(page,item))
#selecting inside this page the desired product
random_product = random_response['results'][item] 
df = pd.DataFrame(random_product.values(), index=random_product.keys(), columns=['data product '+str(random+1)]).transpose()
display(df.style)    

Requested url: http://api.socib.es/data-products/?page=19

Data-product on page 19 at position 3 is described by at least 3 properties:


Knowing a given product id it is also possible to access the above information and even more with the DATA-PRODUCTS/ID/ ENDPOINT:

In [82]:
detailed_end_point = '/data-products/'+random_product['id'] 
detailed_request = requests.get('%s%s' % (api_url, detailed_end_point), headers=headers)
detailed_response = json.loads(detailed_request.text)
print('Requested url: '+'%s%s' % (api_url, detailed_end_point))
values = []
for val in detailed_response.values():
    if type(val) is list:
        ids = []
        for deployment in val:
            ids.append(deployment['id'])
        values.append(', '.join(ids))
    else:
        values.append(val)
df = pd.DataFrame(values, index=[['data-sources ids']+detailed_response.keys()[1:len(detailed_response.keys())]], columns=['data product '+str(random)]).transpose()
display(df.style)  
print('From sources, it is understood that the above product is composed by %s deployments or data-sources'%(len(ids)))

Requested url: http://api.socib.es/data-products/station-galfi


From sources, it is understood that the above product is composed by 2 deployments or data-sources


As we know from <a href="https://github.com/pazrg/SOCIB_API/blob/master/data_sources/finding_your_data_source.ipynb" target="_blank">finding_your_data_source</a>, every data-source is defined by a 14 properties:

In [83]:
print('Each deployment is defined by %s properties:' %(len(detailed_response['sources'][0].keys())))
for prop in detailed_response['sources'][0].keys():
    print('    '+prop)

Each deployment is defined by 15 properties:
    status
    description
    data_type
    instrument
    variables
    available_entries_processing_levels
    platform
    update_datetime
    initial_datetime
    end_datetime
    entries
    coverage_bounding_box
    type
    id
    name


Let's just access some of them to know what are we looking at:

In [84]:
props = ['description','data_type','instrument','platform','initial_datetime','end_datetime']

In [85]:
array = []
ids = []
for val in detailed_response.values():
    if type(val) is list:
        for deployment in val:
            values = []
            for key in deployment.keys():
                if key in props:
                    values.append(deployment[key])
            array.append(values)
            ids.append(deployment['id'])
df = pd.DataFrame(array, index=ids, columns=props)
display(df.style) 

So this would be the case of a reaserch vessel campaing where on board CTD, ADCP, GPS, TS, WS recorded a number of variables and also during which it was released a surface drifter.

Later, with the id of the different ata sources (DATA_SOURCES/ID/DATA) or just exploring its 'entries' (netCDFs) yoiu can work with its data. See how at:<ul><li><a href="https://github.com/pazrg/SOCIB_API/blob/master/data_sources/straightforward_data_access_for_data_sources.ipynb" target="_blank">API & DATA-SOURCE/ID/DATA ENPOINT</a></li><li><a href="https://github.com/pazrg/SOCIB_API/blob/master/data_sources/working_with_data_sources_netcdfs.ipynb" target="_blank">API & THREDDS netCDFs</a></li></ul> 

<h3> DATA-PRODUCTS PARAMS</h3>

SOCIB API enables users to setup certain conditions to match for DATA-PRODUCT filtering so that, from the generic answer, it is possible  to keep only the ones that better suits user needs. These conditions can be built via the so-called PARAMS. PARAMS are additional url elements that allows to filter the initial/bare ENDPOINT answer. These params can be added after a given generic/bare ENDPOINT alone or in combination (more than one):

There are a total of 11 PARAMS you can play with in order to filter the /DATA-PRODUCTS/ ENDPOINT initial answer. All of them can be found next:
<ul><li>initial_datetime</li> 
    <ul>UTC time as YYY-MM-DDTHH:MM:SS</ul>
</ul>
<ul><li>end_datetime</li> 
    <ul>UTC time as YYY-MM-DDTHH:MM:SS</ul>
</ul>
<ul><li>standard_variable</li> 
    <ul>Any value of those returned by <a href="https://github.com/pazrg/SOCIB_API/blob/master/tips/quick_start.ipynb" target="_blank">/standard-variables/ ENDPOINT</a></ul>
</ul>
<ul><li>bbox</li>
    <ul>Area where a given deployment has been operating since deployed (released or placed). Area should be specify as a 4 comma-separated float numbers following the structure: min. lat., max. lat., min. lon., max. lon</ul>
</ul>
<ul><li>platform_type</li>
    <ul>Any value of those returned by <a href="https://github.com/pazrg/SOCIB_API/blob/master/tips/quick_start.ipynb" target="_blank">/platform-types/ ENDPOINT</a></ul>
</ul>
<ul><li>instrument_type</li>
    <ul>Any value of those returned by <a href="https://github.com/pazrg/SOCIB_API/blob/master/tips/quick_start.ipynb" target="_blank">/instrument-types/ ENDPOINT</a></ul>
</ul>
<ul><li>data_type</li>
    <ul>Any value of those returned by <a href="https://github.com/pazrg/SOCIB_API/blob/master/tips/quick_start.ipynb" target="_blank">/data-types/ ENDPOINT</a></ul>
</ul>
<ul><li>status</li>
    <ul>Two posibilities: active or completed</ul>
</ul>
<ul><li>name</li> 
    <ul>Filters the products which contains the given text in its name or description (Case insensitive)</ul>
</ul>
<ul><li>detail</li>
    <ul>"full" or "summary"</ul>
</ul>
<ul><li>page</li>
    <ul>A pagination of 8 is currently set-up when retrieveing the list of data-sources. When <i>count</i> is more than 8 you should loop over the different pages (1 as default) in order to access all deployments details. </ul>
</ul>

Let's find all fixed stations (undertanding such as those where a Weather Station, Coastal Station, HF-Radar, Oceanographic Buoy or Sea Level is involved as platform) that happened in an specific time window: 

In [100]:
#several PARAM examples
counter = []
platform_types = ['Coastal Station','HF-Radar','Oceanographic Buoy', 'Sea Level', 'Weather Station']
for platform_type in platform_types:
    end_point = '/data-products/'
    PARAMS = ['platform_type', 'initial_datetime','end_datetime']
    PARAMS_values = [platform_type,'2016-03-01T00:00:00','2016-06-01T00:00:00']

    #concatenation of PARAMS
    param_concatenation = params_concatenation(PARAMS,PARAMS_values)

    #request with more than one PARAM
    request_surf = requests.get('%s%s?%s' % (api_url, end_point,param_concatenation), headers=headers)
    response_surf = json.loads(request_surf.text)

    counter.append(response_surf['count'])
    print('* Requested url: '+('%s%s?%s' % (api_url, end_point,param_concatenation)).replace (" ","%20"))
print(' ')
print('From '+PARAMS_values[1] +' to '+ PARAMS_values[2]+' SOCIB offers %s products involving Fixed Stations:' %(sum(counter)))
for platform_type, count in zip(platform_types, counter):
    print('         '+platform_type +'('+ str(counter[platform_types.index(platform_type)])+')')

* Requested url: http://api.socib.es/data-products/?platform_type=Coastal%20Station&initial_datetime=2016-03-01T00:00:00&end_datetime=2016-06-01T00:00:00
* Requested url: http://api.socib.es/data-products/?platform_type=HF-Radar&initial_datetime=2016-03-01T00:00:00&end_datetime=2016-06-01T00:00:00
* Requested url: http://api.socib.es/data-products/?platform_type=Oceanographic%20Buoy&initial_datetime=2016-03-01T00:00:00&end_datetime=2016-06-01T00:00:00
* Requested url: http://api.socib.es/data-products/?platform_type=Sea%20Level&initial_datetime=2016-03-01T00:00:00&end_datetime=2016-06-01T00:00:00
* Requested url: http://api.socib.es/data-products/?platform_type=Weather%20Station&initial_datetime=2016-03-01T00:00:00&end_datetime=2016-06-01T00:00:00
 
From 2016-03-01T00:00:00 to 2016-06-01T00:00:00 SOCIB offers 19 products involving Fixed Stations:
         Coastal Station(5)
         HF-Radar(1)
         Oceanographic Buoy(2)
         Sea Level(6)
         Weather Station(5)


In [101]:
#getting the a quick view of the above products
platform_types = ['Coastal Station','HF-Radar','Oceanographic Buoy', 'Sea Level', 'Weather Station']
ids = []
values = []
for platform_type in platform_types:
    end_point = '/data-products/'
    PARAMS = ['platform_type', 'initial_datetime','end_datetime']
    PARAMS_values = [platform_type,'2016-03-01T00:00:00','2016-06-01T00:00:00']
    #concatenation of PARAMS
    param_concatenation = params_concatenation(PARAMS,PARAMS_values)
    a = '%s%s?%s' % (api_url, end_point,param_concatenation) 
    while a != None:
        #requesting every data product to check its summary properties
        request_prof = requests.get(a, headers=headers)
        response_prof = json.loads(request_prof.text)
        numb = 1
        for product in response_prof['results']:
            values.append(product.values())
            ids.append(product['id'])
        a = response_prof['next']
df = pd.DataFrame(values, index=ids, columns=product.keys())
display(df.style)

Let's again have a more detailed look:

In [107]:
#getting the a quick view of the above products
platform_types = ['Coastal Station','HF-Radar','Oceanographic Buoy', 'Sea Level', 'Weather Station']
for platform_type in platform_types:
    props = ['description','data_type','instrument','platform','initial_datetime','end_datetime']
    for product in response_prof['results']:
        if product['id'] != "station-parcbit":
            detailed_end_point = '/data-products/'+product['id'] 
            detailed_request = requests.get('%s%s' % (api_url, detailed_end_point), headers=headers)
            detailed_response = json.loads(detailed_request.text)
            array = []
            ids = []
            for val in detailed_response.values():
                if type(val) is list:
                    for deployment in val:
                        values = []
                        for key in deployment.keys():
                            if key in props:
                                values.append(deployment[key])
                        array.append(values)
                        ids.append(deployment['id'])
            print('Data-sources composing product with id %s'%(product['id']))
            df = pd.DataFrame(array, index=ids, columns=props)
            display(df.style) 

Data-sources composing product with id station-esporles


Data-sources composing product with id station-estellencs


Data-sources composing product with id station-galfi


Data-sources composing product with id station-salines


Data-sources composing product with id station-esporles


...

Data-sources composing product with id station-salines


Data-sources composing product with id station-esporles


Data-sources composing product with id station-estellencs


Data-sources composing product with id station-galfi


Data-sources composing product with id station-salines
